In [1]:
from siphon.catalog import TDSCatalog
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import pandas as pd
from datetime import date, timedelta


In [ ]:
sdate = date(2019,6,13)   # start date
edate = date(2021,10,1)   # end date
dates_to_pull = [x.strftime('%Y%m%d') for x in pd.date_range(sdate,edate-timedelta(days=1),freq='d',)]

In [ ]:
lat_long_lookup = {
    'taipei': [25, 121.5], 
    'la': [34, 118],
    'delhi': [28.5, 77]
}

In [4]:
catUrl = "https://rda.ucar.edu/thredds/catalog/files/g/ds084.1/2017/20170101/catalog.xml";
datasetName = "gfs.0p25.2017010100.f012.grib2";

In [5]:
vars_to_pull = [
 'Albedo_surface_6_Hour_Average',
 'Cloud_water_entire_atmosphere_single_layer',
 'Convective_available_potential_energy_surface',
 'Convective_inhibition_surface',
 'Convective_precipitation_surface_6_Hour_Accumulation',
 'Geopotential_height_surface',
 'Geopotential_height_highest_tropospheric_freezing',
 'Geopotential_height_zeroDegC_isotherm',
 'Geopotential_height_maximum_wind',
 'Geopotential_height_tropopause',
 'Haines_index_surface',
 'ICAO_Standard_Atmosphere_Reference_Height_maximum_wind',
 'ICAO_Standard_Atmosphere_Reference_Height_tropopause',
 'Ice_cover_surface',
 'Land_cover_0__sea_1__land_surface',
 'Latent_heat_net_flux_surface_6_Hour_Average',
 'Momentum_flux_u-component_surface_6_Hour_Average',
 'Precipitable_water_entire_atmosphere_single_layer',
 'Precipitation_rate_surface_6_Hour_Average',
 'Pressure_low_cloud_bottom_6_Hour_Average',
 'Pressure_middle_cloud_bottom_6_Hour_Average',
 'Pressure_high_cloud_bottom_6_Hour_Average',
 'Pressure_low_cloud_top_6_Hour_Average',
 'Pressure_middle_cloud_top_6_Hour_Average',
 'Pressure_high_cloud_top_6_Hour_Average',
 'Pressure_surface',
 'Pressure_maximum_wind',
 'Pressure_convective_cloud_bottom',
 'Pressure_convective_cloud_top',
 'Pressure_tropopause',
 'Pressure_reduced_to_MSL_msl',
 'Relative_humidity_zeroDegC_isotherm',
 'Relative_humidity_entire_atmosphere_single_layer',
 'Relative_humidity_highest_tropospheric_freezing',
 'Sensible_heat_net_flux_surface_6_Hour_Average',
 'Snow_depth_surface',
 'Temperature_low_cloud_top_6_Hour_Average',
 'Temperature_middle_cloud_top_6_Hour_Average',
 'Temperature_high_cloud_top_6_Hour_Average',
 'Temperature_surface',
 'Temperature_maximum_wind',
 'Temperature_tropopause',
 'Total_cloud_cover_low_cloud_6_Hour_Average',
 'Total_cloud_cover_middle_cloud_6_Hour_Average',
 'Total_cloud_cover_high_cloud_6_Hour_Average',
 'Total_cloud_cover_boundary_layer_cloud_6_Hour_Average',
 'Total_cloud_cover_entire_atmosphere_6_Hour_Average',
 'Total_cloud_cover_convective_cloud',
 'Total_ozone_entire_atmosphere_single_layer',
 'Total_precipitation_surface_6_Hour_Accumulation',
 'Categorical_Rain_surface_6_Hour_Average',
 'Categorical_Freezing_Rain_surface_6_Hour_Average',
 'Categorical_Ice_Pellets_surface_6_Hour_Average',
 'Categorical_Snow_surface_6_Hour_Average',
 'Convective_Precipitation_Rate_surface_6_Hour_Average',
 'Potential_Evaporation_Rate_surface',
 'Vertical_Speed_Shear_tropopause',
 'Ventilation_Rate_planetary_boundary',
 'MSLP_Eta_model_reduction_msl',
 'Zonal_Flux_of_Gravity_Wave_Stress_surface_6_Hour_Average',
 'Meridional_Flux_of_Gravity_Wave_Stress_surface_6_Hour_Average',
 'Planetary_Boundary_Layer_Height_surface',
 'Downward_Short-Wave_Radiation_Flux_surface_6_Hour_Average',
 'Upward_Short-Wave_Radiation_Flux_surface_6_Hour_Average',
 'Upward_Short-Wave_Radiation_Flux_atmosphere_top_6_Hour_Average',
 'Downward_Long-Wave_Radp_Flux_surface_6_Hour_Average',
 'Upward_Long-Wave_Radp_Flux_surface_6_Hour_Average',
 'Upward_Long-Wave_Radp_Flux_atmosphere_top_6_Hour_Average',
 'Cloud_Work_Function_entire_atmosphere_single_layer_6_Hour_Average',
 'Sunshine_Duration_surface',
 'Surface_Lifted_Index_surface',
 'Best_4_layer_Lifted_Index_surface',
 'Ground_Heat_Flux_surface_6_Hour_Average',
 'Wilting_Point_surface',
 'Field_Capacity_surface',
 'Water_equivalent_of_accumulated_snow_depth_surface',
 'Water_runoff_surface_6_Hour_Accumulation',
 'Wind_speed_gust_surface',
 'u-component_of_wind_planetary_boundary',
 'u-component_of_wind_maximum_wind',
 'u-component_of_wind_tropopause',
 'v-component_of_wind_planetary_boundary',
 'v-component_of_wind_maximum_wind',
 'v-component_of_wind_tropopause']

In [ ]:
saved_values = {key: [] for key in vars_to_pull}
saved_values['city'] = []
saved_values['date'] = []


for date_str in dates_to_pull:
    print(date_str)
    year = date_str[0:4]
    catUrl = f"https://rda.ucar.edu/thredds/catalog/files/g/ds084.1/{year}/{date_str}/catalog.xml";
    datasetName = f"gfs.0p25.{date_str}00.f024.grib2";
    catalog = TDSCatalog(catUrl)
    ds = catalog.datasets[datasetName]
    dataset = ds.remote_access()
    for city in lat_long_lookup:
        city_lat = lat_long_lookup[city][0]
        city_long = lat_long_lookup[city][1]
        ds_latitudes = dataset.variables['lat'][:]
        ds_longitudes = dataset.variables['lon'][:]
        city_lat_index = np.where(ds_latitudes==city_lat)[0][0]
        city_long_index = np.where(ds_longitudes==city_long)[0][0]
        for var in vars_to_pull:
            try:
                var_value = surface_temp = dataset.variables[var][:][0][city_lat_index, city_long_index]
            except:
                print('missing: {}, {}, {}'.format(date_str, city, var))
                var_val = np.nan
                      
            saved_values[var].append(var_value)
        saved_values['city'].append(city)
        saved_values['date'].append(date_str[0:4] + '-' + date_str[4:6] + '-' + date_str[6:8])
        

20190613
missing: 20190613, taipei, Convective_precipitation_surface_6_Hour_Accumulation
missing: 20190613, taipei, Total_precipitation_surface_6_Hour_Accumulation
missing: 20190613, la, Convective_precipitation_surface_6_Hour_Accumulation
missing: 20190613, la, Total_precipitation_surface_6_Hour_Accumulation
missing: 20190613, delhi, Convective_precipitation_surface_6_Hour_Accumulation
missing: 20190613, delhi, Total_precipitation_surface_6_Hour_Accumulation
20190614
missing: 20190614, taipei, Convective_precipitation_surface_6_Hour_Accumulation
missing: 20190614, taipei, Total_precipitation_surface_6_Hour_Accumulation
missing: 20190614, la, Convective_precipitation_surface_6_Hour_Accumulation
missing: 20190614, la, Total_precipitation_surface_6_Hour_Accumulation
missing: 20190614, delhi, Convective_precipitation_surface_6_Hour_Accumulation
missing: 20190614, delhi, Total_precipitation_surface_6_Hour_Accumulation
20190615
missing: 20190615, taipei, Convective_precipitation_surface_6_H

In [31]:
forecast_data = pd.DataFrame(saved_values)

In [51]:
# forecast_data.to_csv("./forecast_data.csv")